## Using LangChain Tools and Agents to do a WebSearch and Math Calculation

#### 1.  Declare a tool for Searching the internet

In [18]:
from langchain.agents import load_tools
from langchain import SerpAPIWrapper
from langchain.agents import Tool
import os

search = SerpAPIWrapper()
websearch_tool = Tool(
    name = "Search",
    func=search.run,
    description="Use this to lookup information from google search \
                engine",
)

#### 2.  Declare a tool for doing Math Calcualtion

In [25]:
from langchain import LLMMathChain
import boto3
from langchain.llms import Bedrock

math_llm = Bedrock(model_id="anthropic.claude-instant-v1",
                         model_kwargs={"temperature":0,"stop_sequences" : ["```output"]})

llm_math_chain = LLMMathChain(llm=math_llm, verbose=True)

llm_math_chain.llm_chain.prompt.template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

math_tool = Tool(
    name = "Calculator",
    func=llm_math_chain.run,
    description="Useful for when you need to answer questions about math",
)

/Users/thandavm/anaconda3/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


#### 3.  Add tools in a tool Array

In [26]:

tools = [websearch_tool, math_tool]

#### 4.  Create an Agent and add the tools as actions for the Agent

In [27]:


from langchain.agents import initialize_agent
from langchain.agents import AgentType

model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
react_agent_llm = Bedrock(model_id="anthropic.claude-instant-v1", model_kwargs=model_parameter)

react_agent = initialize_agent(tools, 
                               react_agent_llm, 
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True,
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                            #    handle_parsing_errors=True,
                               )

#### 5.  Invoke the Agent

In [28]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["Search", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template



In [29]:
#question = 'who is the president of America'
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"
react_agent(question)



> Entering new AgentExecutor chain...
 Here is the step-by-step process:

Question: What is Amazon SageMaker? What is the launch year multiplied by 2

Thought: I need to search online to find out what Amazon SageMaker is and use a calculator to multiply the launch year by 2.

Action: Search  
Action Input: "Amazon SageMaker"

Observation: ['Amazon SageMaker is a cloud based machine-learning platform that enables developers to create, train, and deploy machine-learning models on the cloud. It also enables developers to deploy ML models on embedded systems and edge-devices. SageMaker was launched in November 2017.', 'Amazon SageMaker kgmid: /g/11h3yhqxn_.', 'Amazon SageMaker initial_release_date: November 29, 2017.', 'Amazon SageMaker developer: Amazon Web Services, Amazon.com.', "High-performance, low-cost ML at scale. Amazon SageMaker is built on Amazon's two decades of experience developing real-world ML applications, ..."]
Thought: Now I know what Amazon SageMaker is. Next I need t

{'input': 'What is Amazon SageMaker? What is the launch year multiplied by 2',
 'output': 'Amazon SageMaker is a cloud based machine-learning platform launched by Amazon Web Services in November 2017. The launch year multiplied by 2 is 4034.'}